In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# read functions.
import os, random
for f in os.listdir('../fun/'): exec(open('../fun/'+f).read())
del f

# Load data
load( '../out/d3-fight-level-transform.pkl' )
print( X.shape )

random.seed( 1207 )

(3355, 164)


In [2]:
# train test split.
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(
    X, y, 
    random_state = 744,
    test_size = 0.3
)

In [7]:
# run grid search on parameters.    
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
m = AdaBoostClassifier()
readpriorgrid = False
if ( not readpriorgrid  )  or ( 'grid-randomforest.pkl' not in os.listdir('../out/') ):
    grid_adaboost = GridSearchCV(
        m, cv = 6, scoring = 'precision', n_jobs = -1, verbose = True,
        param_grid = dict(
            n_estimators = range( 40, 80, 5 ),
            learning_rate  = [ x/100 for x in range( 80, 120, 5 ) ],
            algorithm = ['SAMME', 'SAMME.R']
        )
    )
    grid_adaboost.fit( X_train, y_train )
    save( '../out/grid-adaboost.pkl', grid_adaboost )
else:
    load( '../out/grid-adaboost.pkl' )
print(grid_adaboost.best_params_)
print(grid_adaboost.best_score_)
m = grid_adaboost.best_estimator_

Fitting 6 folds for each of 128 candidates, totalling 768 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 768 out of 768 | elapsed:  3.2min finished


{'algorithm': 'SAMME.R', 'learning_rate': 0.9, 'n_estimators': 45}
0.6598357913703452


In [9]:
# Fit model and check in-model fit.
from sklearn.metrics import f1_score, precision_score, recall_score
m.fit( X_train, y_train )
p = m.predict(X_train)
print({ 
    'fscore': f1_score( y_train, p ), 
    'precision': precision_score( y_train, p ), 
    'recall': recall_score( y_train, p ) 
})

{'fscore': 0.7829432510420007, 'precision': 0.7333333333333333, 'recall': 0.8397524071526823}


In [10]:
# accuracy against test data.
p = m.predict(X_test)
print( { 
    'fscore': f1_score( y_test, p ), 
    'precision': precision_score( y_test, p ), 
    'recall': recall_score( y_test, p ) 
} )

{'fscore': 0.7267311988086375, 'precision': 0.7001434720229556, 'recall': 0.7554179566563467}


In [11]:
from sklearn.model_selection import cross_validate
kfold = cross_validate( 
    m, X, y, 
    cv = 10, 
    scoring = [ 'precision', 'recall' ], 
    n_jobs = -1 
)
print( 
  'Mean precision: %s \nMean recall %s' % 
  ( kfold['test_precision'].mean(), kfold['test_recall'].mean() ) 
)

Mean precision: 0.5681560561112227 
Mean recall 0.6071428571428571


f1-score in-model is higher than on test indicating **over-fitting**. 

Test-set precision is still not great at 69% (predicting all-wins give 60%).